
## Overview

This notebook shows you how to create and query a table or DataFrame loaded from data stored in AWS S3. There are two ways to establish access to S3: [IAM roles](https://docs.databricks.com/user-guide/cloud-configurations/aws/iam-roles.html) and access keys.

*We recommend using IAM roles to specify which cluster can access which buckets. Keys can show up in logs and table metadata and are therefore fundamentally insecure.* If you do use keys, you'll have to escape the `/` in your keys with `%2F`.

This is a **Python** notebook so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` magic command. Python, Scala, SQL, and R are all supported.

In [0]:
dbutils.fs.rm("/FileStore/tables/RangeText.txt")

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-4160685445483543>:1
----> 1 dbutils.fs.rm("/FileStore/tables/RangeText.txt")

File /databricks/python_shell/dbruntime/dbutils.py:364, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    362 exc.__context__ = None
    363 exc.__cause__ = None
--> 364 raise exc

ExecutionError: An error occurred while calling o406.rm.
: com.databricks.rpc.UnknownRemoteException: Remote exception occurred:
	org.apache.hadoop.fs.PathIsNotEmptyDirectoryException: `s3a://databricks-prod-storage-oregon/devtierprod1/2522566250873381/FileStore/tables/RangeText.txt': Directory is not empty
		at shaded.databricks.org.apache.hadoop.fs.s3a.impl.DeleteOperation.execute(DeleteOperation.java:273)
		at shaded.databricks.org.apache.hadoop.fs.s3a.S3AFileSystem.deleteWithInstrumentation(S3AFileSystem.java:34

In [0]:
%fs
ls FileStore/tables/

path,name,size,modificationTime
dbfs:/FileStore/tables/2010_summary.csv,2010_summary.csv,7121,1728547018000
dbfs:/FileStore/tables/NewFile/,NewFile/,0,0
dbfs:/FileStore/tables/RangeFile/,RangeFile/,0,0
dbfs:/FileStore/tables/RangeText/,RangeText/,0,0
dbfs:/FileStore/tables/RangeText.txt/,RangeText.txt/,0,0
dbfs:/FileStore/tables/SparkText,SparkText,1761,1728552324000
dbfs:/FileStore/tables/SparkText.txt,SparkText.txt,513,1728828118000


#### Reading the file via DataframeReader API

We can load the data in spark via `DataframeReader API` and can also write it via `DataframeWriter API`.
To access DataframeReaderAPI we have to use `spark.read()` method. 

Parameters required while reading any file from `spark.read()` :
1. **format (optional) :** Data file format such as csv, json, jdbc/odbc, table *(default format is parquet)*
2. **option (optional) :** To set up different options for file reading such as *inferschema, mode, header, path*
3. **schema (optional) :** To pass manual schema
4. **load (required) :** Path where our data is residing 

##### Reading Modes in PySpark 
In PySpark, when reading data into a DataFrame from external sources, you can specify a reading mode to control how the system should handle issues such as missing files, corrupt records, and schema mismatches. The available reading modes depend on the data source. 
1. **FAILFAST MODE :** This mode fails the reading process if it encounters any malformed/corrupted data or schema mismatch.
2. **DROPMALFORMED MODE :** This mode drops any row that contains malformed data (e.g., extra columns).
3. **PARMISSVE MODE :** This mode is the default mode while reading the dataframe. In permissive mode, PySpark reads as much data as possible and stores corrupt records in a `“_corrupt_record”` column ans set the `null value` to all the corrupted fields.



In [0]:
# File location and type
file_location = "/FileStore/tables/2010_summary.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df.head(5))

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,1
United States,Ireland,264
United States,India,69
Egypt,United States,24
Equatorial Guinea,United States,1


In [0]:
# Create a view or table

temp_table_name = "test_flight_table"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `test_flight_table` limit 10

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,1
United States,Ireland,264
United States,India,69
Egypt,United States,24
Equatorial Guinea,United States,1
United States,Singapore,25
United States,Grenada,54
Costa Rica,United States,477
Senegal,United States,29
United States,Marshall Islands,44


In [0]:
# Since this table is registered as a temp view, it will only be available to this notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "flight_table"

df.write.mode('overwrite').format('delta').saveAsTable(permanent_table_name)

In [0]:
%sql

select * from flight_table limit 10

DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,1
United States,Ireland,264
United States,India,69
Egypt,United States,24
Equatorial Guinea,United States,1
United States,Singapore,25
United States,Grenada,54
Costa Rica,United States,477
Senegal,United States,29
United States,Marshall Islands,44


In [0]:
df = spark.table('samples.nyctaxi.trips')
df.show(5)
print(df.printSchema())

+--------------------+---------------------+-------------+-----------+----------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|trip_distance|fare_amount|pickup_zip|dropoff_zip|
+--------------------+---------------------+-------------+-----------+----------+-----------+
| 2016-02-14 16:52:13|  2016-02-14 17:16:04|         4.94|       19.0|     10282|      10171|
| 2016-02-04 18:44:19|  2016-02-04 18:46:00|         0.28|        3.5|     10110|      10110|
| 2016-02-17 17:13:57|  2016-02-17 17:17:55|          0.7|        5.0|     10103|      10023|
| 2016-02-18 10:36:07|  2016-02-18 10:41:45|          0.8|        6.0|     10022|      10017|
| 2016-02-22 14:14:41|  2016-02-22 14:31:52|         4.51|       17.0|     10110|      10282|
+--------------------+---------------------+-------------+-----------+----------+-----------+
only showing top 5 rows

root
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- trip_dis